# Langchain crash course

In [1]:
# from secret_key import openapi_key
# import os
# os.environ['OPENAI_API_KEY'] = openapi_key

import os
os.environ['OPENAI_API_KEY'] = "" #add your key here 

## LLMs

In [3]:
!pip install -U langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 949.8/949.8 kB 12.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.0/291.0 kB 15.8 MB/s eta 0:00:00

[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [4]:
from langchain_openai import OpenAI

llm = OpenAI(temperature=0.9)
name = llm.predict("I want to open a restaurant for Indian food. Suggest a fency name for this.")
print(name)



"Spice Palace"


In [5]:
llm("I want to open a restaurant for Indian food. Suggest a fency name for this.")

/Users/Parzon/Downloads/GenAI/langchain/venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'\n\n"Spice Palace" '

## Prompt Templates

In [9]:
from langchain.prompts import PromptTemplate

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)
p = prompt_template_name.format(cuisine="Italian")
print(p)

# print(llm.predict(p))

I want to open a restaurant for Italian food. Suggest a fency name for this.


In [7]:
print(llm.predict(p))



"Bellissimo Bistro"


## Chains

In [8]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt_template_name)
chain.run("Mexican")

/Users/Parzon/Downloads/GenAI/langchain/venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'\n\n"El Jardín de Sabor" (The Garden of Flavor)'

In [9]:
chain = LLMChain(llm=llm, prompt=prompt_template_name, verbose=True)
chain.run("Mexican")



> Entering new LLMChain chain...
Prompt after formatting:
I want to open a restaurant for Mexican food. Suggest a fency name for this.

> Finished chain.


'\n\n"La Casa de Sabores"  (The House of Flavors)'

In [11]:
llm = OpenAI(temperature=0.6)

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)

name_chain =LLMChain(llm=llm, prompt=prompt_template_name)

prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template="""Suggest some menu items for {restaurant_name}"""
)

food_items_chain = LLMChain(llm=llm, prompt=prompt_template_items)

#### Simple Sequential Chain

In [12]:
from langchain.chains import SimpleSequentialChain
chain = SimpleSequentialChain(chains = [name_chain, food_items_chain])

content = chain.run("Indian")
print(content)



1. Spicy Chicken Tikka Masala
2. Lamb Vindaloo
3. Vegetable Korma
4. Tandoori Shrimp
5. Samosas (vegetable or chicken)
6. Naan bread (plain or garlic)
7. Mango Lassi
8. Chana Masala (chickpea curry)
9. Butter Chicken
10. Aloo Gobi (potato and cauliflower curry)
11. Chicken Biryani
12. Palak Paneer (spinach and cheese curry)
13. Tandoori Mixed Grill (chicken, lamb, and shrimp)
14. Vegetable Pakora (mixed vegetable fritters)
15. Gulab Jamun (traditional Indian dessert)


#### Sequential Chain

In [13]:
llm = OpenAI(temperature=0.7)

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)

name_chain =LLMChain(llm=llm, prompt=prompt_template_name, output_key="restaurant_name")

In [14]:
llm = OpenAI(temperature=0.7)

prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template="Suggest some menu items for {restaurant_name}."
)

food_items_chain =LLMChain(llm=llm, prompt=prompt_template_items, output_key="menu_items")

In [15]:
from langchain.chains import SequentialChain

chain = SequentialChain(
    chains = [name_chain, food_items_chain],
    input_variables = ['cuisine'],
    output_variables = ['restaurant_name', "menu_items"]
)

In [16]:
chain({"cuisine": "Indian"})

/Users/Parzon/Downloads/GenAI/langchain/venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


{'cuisine': 'Indian',
 'restaurant_name': '\n\n"Spice Palace"',
 'menu_items': '\n\n1. Spicy Tandoori Chicken\n2. Vegetable Samosas\n3. Lamb Vindaloo\n4. Butter Chicken\n5. Aloo Gobi (spiced cauliflower and potatoes)\n6. Chicken Tikka Masala\n7. Vegetable Biryani\n8. Mango Lassi (spiced mango yogurt drink)\n9. Garlic Naan\n10. Chana Masala (spiced chickpea curry)\n11. Tandoori Shrimp\n12. Palak Paneer (spiced spinach and cheese)\n13. Chicken Korma\n14. Dal Makhani (creamy spiced lentils)\n15. Gulab Jamun (spiced milk balls in syrup)'}

## Agents

In [3]:
# make sure yoy have installed this package: pip install google-search-results
# from secret_key import serpapi_key
# os.environ['SERPAPI_API_KEY'] = serpapi_key

os.environ['SERPAPI_API_KEY'] = "" #add serpapi key here

#### serpapi and llm-math tool

In [4]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain_openai import OpenAI

llm = OpenAI(temperature=0)

# The tools we'll give the Agent access to. Note that the 'llm-math' tool uses an LLM, so we need to pass that in.
tools = load_tools(["serpapi", "llm-math"], llm=llm)

# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# Let's test it out!
agent.run("What was the GDP of US in 2022 plus 5?")

/Users/Parzon/Downloads/GenAI/langchain/venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(
/Users/Parzon/Downloads/GenAI/langchain/venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
 I should use a calculator to add 5 to the GDP of US in 2022
Action: Calculator
Action Input: GDP of US in 2022 + 5

ValueError: LLMMathChain._evaluate("
GDP_US_2022 + 5
") raised error: 'GDP_US_2022'. Please try again with a valid numerical expression

In [5]:
agent.run("What is 10 plus 5?")



> Entering new AgentExecutor chain...
 I should use a calculator to solve this math problem.
Action: Calculator
Action Input: 10 + 5
Observation: Answer: 15
Thought: I now know the final answer.
Final Answer: 15

> Finished chain.


'15'

#### Wikipedia and llm-math tool

In [6]:
# install this package: pip install wikipedia

# The tools we'll give the Agent access to. Note that the 'llm-math' tool uses an LLM, so we need to pass that in.
tools = load_tools(["wikipedia", "llm-math"], llm=llm)

# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(
    tools, 
    llm, 
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
    verbose=True
)

# Let's test it out!
agent.run("When was Elon musk born? What is his age right now in 2023?")



> Entering new AgentExecutor chain...
 I should use Wikipedia to find information about Elon Musk's birthdate and age.
Action: Wikipedia
Action Input: Elon Musk
Observation: Wikipedia is not a valid tool, try one of [wikipedia, Calculator].
Thought: I should use Wikipedia to find information about Elon Musk's birthdate and age.
Action: Wikipedia
Action Input: Elon Musk
Observation: Wikipedia is not a valid tool, try one of [wikipedia, Calculator].
Thought: I should use Wikipedia to find information about Elon Musk's birthdate and age.
Action: Wikipedia
Action Input: Elon Musk
Observation: Wikipedia is not a valid tool, try one of [wikipedia, Calculator].
Thought: I should use Calculator to calculate Elon Musk's age in 2023.
Action: Calculator
Action Input: 2023 - 1971
Observation: Answer: 52
Thought: I now know the final answer.
Final Answer: Elon Musk was born on June 28, 1971, so his age in 2023 will be 52.

> Finished chain.


'Elon Musk was born on June 28, 1971, so his age in 2023 will be 52.'

## Memory

In [10]:
from langchain.chains import LLMChain
chain = LLMChain(llm=llm,prompt=prompt_template_name)
name = chain.run("Mexican")
print(name)



"El Sabor Exquisito" (The Exquisite Flavor)


In [11]:
name = chain.run("Indian")
print(name)



"Spice Palace"


In [12]:
chain.memory

In [13]:
type(chain.memory)

NoneType

#### ConversationBufferMemory

In [14]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

chain = LLMChain(llm=llm, prompt=prompt_template_name, memory=memory)
name = chain.run("Mexican")
print(name)



"El Sabor Exquisito" (The Exquisite Flavor)


In [15]:
name = chain.run("Arabic")
print(name)



"Al-Fawarish" (meaning "The Treasures" in Arabic)


In [16]:
print(chain.memory.buffer)

Human: Mexican
AI: 

"El Sabor Exquisito" (The Exquisite Flavor)
Human: Arabic
AI: 

"Al-Fawarish" (meaning "The Treasures" in Arabic)


#### ConversationChain

In [17]:
from langchain.chains import ConversationChain

convo = ConversationChain(llm=OpenAI(temperature=0.7))
print(convo.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


In [18]:
convo.run("Who won the first cricket world cup?")

' The first cricket World Cup was won by the West Indies in 1975. They beat Australia by 17 runs in the final match.'

In [19]:
convo.run("How much is 5+5?")

'  5+5 is equal to 10.'

In [20]:
convo.run("Who was the captain ofthe winning team?")

' The captain of the winning team was Clive Lloyd. He led the West Indies team to victory with his impressive batting and leadership skills.'

In [21]:
print(convo.memory.buffer)

Human: Who won the first cricket world cup?
AI:  The first cricket World Cup was won by the West Indies in 1975. They beat Australia by 17 runs in the final match.
Human: How much is 5+5?
AI:   5+5 is equal to 10.
Human: Who was the captain ofthe winning team?
AI:  The captain of the winning team was Clive Lloyd. He led the West Indies team to victory with his impressive batting and leadership skills.


#### ConversationBufferWindowMemory

In [25]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=10)#change K to determine the length of conversation memory

convo = ConversationChain(
    llm=OpenAI(temperature=0.7),
    memory=memory
)
convo.run("Who won the first cricket world cup?")

' The first cricket world cup was held in 1975 and was won by the West Indies team. The final match was played between the West Indies and Australia, with the West Indies team securing a victory by 17 runs.'

In [26]:
convo.run("How much is 5+5?")

'  5+5 is equal to 10. This is a basic mathematical equation and the answer is a fundamental concept in mathematics.'

In [27]:
convo.run("Who was the captain of the winning team?")

' The captain of the winning team, the West Indies, was Clive Lloyd. He was a legendary cricketer and led the team to victory in both the first and second cricket world cups. He was also known for his powerful batting and exceptional leadership skills.'